# AWESAM - **A**daptive-**W**indow Volcanic **E**vent **S**election **A**nalysis **M**odule

Awesam is a package containing tools to create seismo-volcanic event catalogs from seismic data. This includes:
- download seismic data from FDSN-datacenters,
- create seismo-volcanic event catalogs,
- compare catalogs from two stations to identify non-volcanic disturbances,
- identification of earthquakes.


For a detailed description of each step see the following publication: [Fenner et al., Automated Seismo-Volcanic Event Detection Applied to Stromboli (Italy), Frontiers in Earth Science, 2022](https://www.doi.org/10.3389/feart.2022.809037)

On Youtube, there is a video tutorial too: [Tutorial (Youtube)]().

The following code shows how to create a catalog for Stromboli (Italy). Afterwards it is explained, how to generalize the code for other volcanoes.

If something does not work as described, please open an issue on [GitHub](https://github.com/srivastavaresearchgroup/AWESAM).

## **Step 1:** Installation

1. Clone the github repository (with `git clone https://github.com/srivastavaresearchgroup/AWESAM.git`). Be sure python can import the package.

2. Install necessary the dependencies: `numpy`, `obspy`, `scipy`, `pandas`, `torch`, and for testing `matplotlib`.

3. Some parts of AWESAM are written in C and have to be compiled first. On systems where `gcc` is installed, this is automatically done. But, if no compiler can be found, the module automatically switches to a (less performant) python-backend. Then, `numba` must be installed too.

## **Step 2:** Imports and Configuration

Before starting, some basic configurations need to be done. There are two possibilities:
1. Edit the file `awesam/config.py` and reload the library (e.g. restart the jupyter notebook)
2. Directly configure the settings in the code (see below)

In [ ]:
from awesam import config

# all downloaded seismic data will be stored here
config.settings["general"]["mseed_directory"] = "/path/to/..."
# the final catalogs will be stored here
config.settings["general"]["catalog_directory"] = "/path/to/..."

# additional steps, that are not necessary for now, and which are explained later
config.settings["general"]["pipeline"]["use_catalog_consolidation"] = False
config.settings["general"]["pipeline"]["use_earthquake_classification"] = False

# Volcano specific settings (in this case: Stromboli)
config.settings["general"]["service"] = "INGV"
config.settings["general"]["stations"] = [
    {"station": "IV.IST3.--", "channels": ["HHN", "HHE", "HHZ"], "sampling_rate": 100},
    {"station": "IV.ISTR.--", "channels": ["HHN", "HHE", "HHZ"], "sampling_rate": 100},
]
config.settings["general"]["coordinates"] = np.array([38.793315, 15.211588])

## **Step 3:** Data Download

In [ ]:
from awesam import MassDownload

# Download data from all stations and all channels.
MassDownload.multiprocessing_download(
    start=dt.datetime(2021, 1, 1), end=dt.datetime(2021, 1, 31)
)

> Note: Multiprocessing is used for optimal performance. This often results in error messages not being displayed (in jupyter notebooks). If something doesn't seem to work, add `use_multiprocessing=False` as additional argument.

## **Step 4:** Catalog Pipeline

Now the main step: The complete catalog is generated for the specified period. In this process multiple files are created in `config.settings['general']['catalog_directory']`:
- `test_[STATION]_events.csv`: Catalog for each station.
- `test_gaps.csv`: A list of gaps in data for all stations.
- `test_FINAL.csv`: If `use_catalog_consolidation` or `use_earthquake_classification` are True, the final catalog contains the combined catalogs from two stations with the additional columns `event_probability` and/or `earthquakae_probability`. If both additional steps are set to False, test_FINAL.csv will be a copy of `test_[STATION]_events.csv` with the first station. 

In [ ]:
from awesam import CatalogPipeline

CatalogPipeline.multiprocessing_pipeline(
    name="test",
    start=dt.datetime(2021, 1, 1),
    end=dt.datetime(2021, 1, 31),
)

# loads the catalog
pd.read_csv(
    sys.path.join(config.settings["general"]["catalog_directory"], "test_FINAL.csv")
)

To visualize the generated catalog in a plot (with the seismic data), you can use the submodule `awesam.testing`:

In [ ]:
from awesam import tests

# creates one test plot with a random date
tests.test_catalog(n=1, name="test", duration=dt.timedelta(hours=1))

By default AWESAM is configured for Stromboli. If something doesn't look right, try to change the following settings. See the file `awesam/config.py` for description of each parameter:

In [ ]:
config.settings["EventDetection"]["threshold_window_size"]
config.settings["EventDetection"]["kernel_factor"]
config.settings["EventDetection"]["downsampling"]
config.settings["EventDetection"]["threshold_factor"]

# To get a more detailed insight what happens, you can use `tests.test_event_det`
# to generate a plot with more information about the event detection process
tests.test_event_det(dt.datetime(2021, 1, 5), dt.timedelta(minutes=30))

## **Step 5:** Catalog Consolidation (optional)

To use Catalog Consolidation, two stations must be available (currently the algorithm only works for exactly two stations). To use it just set `config.settings['general']['pipeline']['use_catalog_consolidation'] = True` and rerun the CatalogPipeline.



In [ ]:
# To configure the algorithm, change.
# - the first element in the array is the factor multiplied with the time difference
# - the last element in the array is the factor multiplied with the amplitude difference
config.settings["CatalogConsolidation"]["default_metric"] = np.array(
    [200, 0, 0, 0.1], dtype=np.double
)

# testing
tests.catalog_consolidation_plot(dt.datetime(2021, 1, 5), dt.timedelta(hours=1))

## **Step 6:** Earthquake Classification (optional)

In many cases, it is useful to know whether an event is volcanic or if it is an earthquake, expecially for large events. 

The EarthquakeClassification algorithm attempts to estimate the arrival time and amplitude of earthquakes at the volcano. This is estimated from an official catalog, for example the ISC catalog. Then it utilizes the CatalogConsolidation algorithm again.

First, download an earthquake catalog for the region and store it in `config.settings['general']['earthquake_catalog_directory']`. Store it as a csv-file per year with the name: `2020.csv`, `2021.csv` etc. It must have the following columns: `date,latitude,longitude,magnitude,depth`. Depth must be in kilometers.

To use it, set `config.settings['general']['use_earthquake_classification']` to true and rerun the pipeline. The resulting catalog has an additional column called “earthquake_probability”. See the settings in `EarthquakeClassification` for details about the configuration. 

## **Step 7:** Customization for other volcanoes

All settings can be changed in the file `config.py`. Each setting is described there. Make sure to reload `config.py` after changing something with `importlib.reload(config)`. There are also presetsfor Stromboli (Italy), Etna (Italy), Whakaari (New Zealand), and Yasur (Whakaari)  in `presets.py` .
